<a href="https://colab.research.google.com/github/santiagogiasone/COVID-PersonLimiter/blob/master/HPC%20(directorio)/Cuaderno_2_grupo17_2021.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<p align="center">
  <img align="center" width="256" height="256" src="https://i.imgur.com/OskOQbI.png">
</p>

#Deepfake Discord Bot


1.   #### Introduccion
Luego de seguir el instructivo correspondiente, usted podra invitar a su servidor de discord al bot creado que dandole como input una video y una imagen, le creara un deepfake de ese video con la cara que aparece en la imagen.





2.   #### Armado del ambiente

Primero que nada es indispensable que configure el BOT de Discord, si usted no sabe como crear un bot y obtener su TOKEN nosotros le proveemos el mismo y puede dejar el TOKEN configurado en la variable de entorno por defecto.

Link de Invitacion al BOT (Por si quiere usar el default): https://discord.com/api/oauth2/authorize?client_id=912791690639704134&permissions=517544069184&scope=bot

Instalamos las dependencias de python para que el proyecto funcione correctamente



**Monta tu google drive dentro del proyecto**

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


**Copiar la siguiente carpeta en el directorio raiz de tu Google Drive**
https://drive.google.com/drive/folders/1TePCM7kivxvupd1vfgPbAY0IoE88zdDj?usp=sharing

**Clonar el Repositorio de Deepfake**

In [ ]:
!git clone https://github.com/AliaksandrSiarohin/motion-cosegmentation motion-co-seg

Cloning into 'motion-co-seg'...
remote: Enumerating objects: 103, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 103 (delta 7), reused 0 (delta 0), pack-reused 90
Receiving objects: 100% (103/103), 42.61 MiB | 30.99 MiB/s, done.
Resolving deltas: 100% (35/35), done.


In [ ]:
cd motion-co-seg/

/content/motion-co-seg


In [ ]:
!pip install discord

     |████████████████████████████████| 786 kB 5.1 MB/s 
     |████████████████████████████████| 1.3 MB 33.6 MB/s 
     |████████████████████████████████| 160 kB 56.7 MB/s 
     |████████████████████████████████| 271 kB 74.4 MB/s 


**Parcheo asyncio**

In [ ]:
## Este módulo parchea asyncio para permitir el uso anidado de asyncio.run y loop.run_until_complete.
import nest_asyncio 
nest_asyncio.apply()

**Dependencias de DeepFake**

In [ ]:
import imageio
from skimage.transform import resize
from part_swap import load_checkpoints
from part_swap import make_video
from skimage import img_as_ubyte

**Variables de entorno**

In [ ]:
## Configuracion de Variables de Entorno

TOKEN = "OTEyNzkxNjkwNjM5NzA0MTM0.YZ1FgQ.OvL3wC5RwW2tAv022qemb9aro_k" #Aqui va su propio TOKEN de su BOT, si no lo creo dejelo asi como esta
TOBEY = "/content/gdrive/My Drive/colab_fakecord/tobey.mp4"
DICAPRIO = "/content/gdrive/My Drive/colab_fakecord/10.mp4"
cpu = False
reconstruction_module, segmentation_module = load_checkpoints(config='config/vox-256-sem-5segments.yaml', 
                                               checkpoint='/content/gdrive/My Drive/colab_fakecord/vox-5segments.pth.tar',
                                               blend_scale=1)

3. #### Desarrollo

Luego de iniciar el ambiente de desarrollo, procedemos a levantar nuestro servicio de discord y crear nuestra funcion para obtener el video con la cara del protagonista del video modificada por la que el usuario de discord envio. Para ello redimensionamos ambos archivos para mantener consistencia en el video y los transformarmos a un tipo de variable aceptable por la libreria de deepfake, luego de esto la libreria de discord se encarga de enviar el mensaje conjunto al video modificado.



**Definimos nuestra funcion de Deepfake**

In [ ]:
async def get_video(img_url, file_name, video_path):
  filename, file_extension = os.path.splitext(file_name)
  r = requests.get(img_url)
  image = imageio.imread(r.content, file_extension)
  image = resize(image, (256, 256))[..., :3]
  video = imageio.mimread(video_path)
  video_final = []
  try:
      for im in video:
          video_final.append(im)
  except RuntimeError:
      pass
  video_final = [resize(frame, (256, 256))[..., :3] for frame in video_final]
  predictions = make_video(swap_index=[1, 2, 5], source_image = image, target_video = video_final,
                           segmentation_module=segmentation_module, reconstruction_module=reconstruction_module, use_source_segmentation=True)
  return predictions

**Corremos el servicio del Bot de Discord**

In [ ]:
import discord
from discord.ext import commands
import requests
import shutil
import os

client = commands.Bot(command_prefix = "#")

@client.event
async def on_ready():
  print("Nos loggeamos como {0.user}".format(client))

@client.command(aliases=['some_dirt','tobey'])
async def dirt(ctx):

  if ctx.message.author == client.user:
    return
  
  if(len(ctx.message.attachments) > 0):
    await ctx.send("Generando Deepfake")
    predictions = await get_video(ctx.message.attachments[0].url, ctx.message.attachments[0].filename, TOBEY)
    newfile_path = f"../generated{ctx.message.author}.mp4"
    if os.path.exists(newfile_path):
      os.remove(newfile_path)
    imageio.mimsave(newfile_path, [img_as_ubyte(frame) for frame in predictions])
    await ctx.send("Deepfake generado", file=discord.File(newfile_path))

@client.command(aliases=['dicaprio','leo'])
async def dicap(ctx):

  if ctx.message.author == client.user:
    return
  
  if(len(ctx.message.attachments) > 0):
    await ctx.send("Generando Deepfake")
    predictions = await get_video(ctx.message.attachments[0].url, ctx.message.attachments[0].filename, DICAPRIO)
    newfile_path = f"../generated{ctx.message.author}.mp4"
    if os.path.exists(newfile_path):
      os.remove(newfile_path)
    imageio.mimsave(newfile_path, [img_as_ubyte(frame) for frame in predictions])
    await ctx.send("Deepfake generado", file=discord.File(newfile_path))

client.run(TOKEN)

ModuleNotFoundError: ignored

5. #### Conclusion

A completar

6. #### Bibliografia

######Librerias de GITHUB sobre Deepfake
https://github.com/AliaksandrSiarohin/first-order-model
https://github.com/AliaksandrSiarohin/motion-cosegmentation
######Informacion de como utilizar la API de Discord:
https://discordpy.readthedocs.io/en/stable/api.html
######Videos Utilizado para el Deepfake:
https://www.youtube.com/watch?v=AITqA6OC9aA 

